실행 위치: Google Colab

# 1. 모델 로드

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "openai-community/gpt2-xl"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    trust_remote_code=True,
).to('cuda')

tokenizer = AutoTokenizer.from_pretrained(model_name)



- 여러 input을 배치 형태로 전달할 경우 tensor 의 크기가 동일해야 함.
- 각 문장을 구성하는 토큰 개수는 서로 다르지만 input 텐서 크기를 맞추기 위해 padding 수행
- 패딩 토큰으로 eos_token (토큰 ID: 50256) 로 수행

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

# 2. 테스트 시퀀스 준비

In [ ]:
inputs = ['Write a short poem about the beauty of a starry night',
          'Tell me a story about a brave adventurer exploring a mysterious forest',
          'Explain how a computer processes data in simple terms',
          'Describe a futuristic city where robots and humans coexist',
          'What are the benefits of daily meditation for mental health?',
          "Tell a funny story about a cat who thinks it's a detective.",
          "Create a dialogue between two friends planning a hiking trip.",
          "How would you convince someone to try a new hobby like painting?",
          "Write a scene where a pirate discovers a hidden treasure.",
          "What is the science behind rainbows in an easy explanation?",
          "Imagine a world where everyone can fly; describe a day in that world."
          ]
print(len(inputs))

In [ ]:
encoded_inputs = tokenizer(inputs, return_tensors='pt', padding=True, truncation=True).to('cuda')
print(encoded_inputs['input_ids'])

# 3. 배치 추론 수행

In [ ]:
import time
start_time = time.time()
outputs = model.generate(
        **encoded_inputs,
        max_length=1000,
        repetition_penalty=2.0,
        num_beams=5,
        no_repeat_ngram_size=2
)
end_time = time.time()

In [ ]:
print(outputs.shape)
print('------')
print(outputs)


## 초당 토큰 수 계산
 - 모델 응답에는 질의 토큰도 포함되어 있으므로 토큰 개수 계산시 질의 내용 토큰 수 빼기 수행

In [ ]:
total_tokens = sum(len(output) - encoded_inputs['input_ids'].shape[1] for output in outputs)
elapsed_time = end_time - start_time

In [ ]:
print(f'{len(inputs)} 개 시퀀스의 총 응답 토큰 수: {total_tokens}')
print(f'응답 소요 시간: {elapsed_time}')
print(f'초당 생성 토큰 수: {total_tokens / elapsed_time}')

# 참고) 출력 내용 사엣보기

In [ ]:
for i, output in enumerate(outputs):
  print(f'----------- {i} -----------')
  print(tokenizer.decode(output))
